In [1]:
%load_ext autoreload
%autoreload 2

import dissociation as ds

import jax
import jax.numpy as np

In [ ]:
in_dim = 6
hidden_dim = 16
out_dim = 2
p = 4

steps = 101

xs, ys = ds.random_regression_task(in_dim, out_dim, p)
xs, ys = ds.random_regression_task(in_dim, out_dim, p)

noise_performance = []
for f in [ds.walk_gls, ds.walk_lss, ds.walk_mrns, ds.walk_mwns]:
    print(f)
    w1s, w2s = f(xs, ys, hidden_dim, steps)
    w2_star = ys @ xs.T @ w1s[0].T @ np.linalg.pinv(w1s[0] @ xs @ xs.T @ w1s[0].T)

    ys_ = np.einsum("oh,thi,in->ton", w2_star, w1s, xs)
    decoding_performance.append(0.5 / p * np.linalg.norm(ys_ - ys, axis=(1, 2)) ** 2)

In [2]:
in_dim = 5
hidden_dim = 7
out_dim = 3
p = 6

w1, w2 = ds.sample_weights(in_dim, hidden_dim, out_dim, 0.2, 0.7)
xs, ys = ds.random_regression_task(in_dim, out_dim, p)

sig_xx = 1.0 / p * xs @ xs.T
sig_yx = 1.0 / p * ys @ xs.T
sig_yy = 1.0 / p * ys @ ys.T

u, s, v = ds.compact_svd(sig_yx @ np.linalg.pinv(sig_xx))
r = jax.random.orthogonal(jax.random.key(1), hidden_dim)[:, : s.shape[0]]
w1 = r @ np.sqrt(s) @ v.T
w2 = u @ np.sqrt(s) @ r.T

sigma_x = 0.05
sigma_1 = 1.0
sigma_2 = 0.5

print(ds.noise_sensitivity(w1, w2, xs, ys, sigma_x, sigma_1, sigma_2, 1000000))
# print(ds.input_noise_sensitivity_theory(w1, w2, xs, ys, sigma))

s1 = np.linalg.norm(w2 @ w1) ** 2
s2 = sigma_1**2 * in_dim * np.linalg.norm(w2) ** 2
s3 = sigma_2**2 * out_dim * np.linalg.norm(w1) ** 2
s4 = sigma_1**2 * in_dim * sigma_2**2 * out_dim * hidden_dim
t1 = sigma_x**2 / 2 * (s1 + s2 + s3 + s4)

s1 = sigma_1**2 * np.linalg.norm(w2) ** 2 * np.linalg.norm(xs) ** 2
s2 = sigma_2**2 * out_dim * np.linalg.norm(w1 @ xs) ** 2
s3 = sigma_1**2 * hidden_dim * out_dim * sigma_2**2 * np.linalg.norm(xs) ** 2
t2 = 1 / (2 * p) * (s1 + s2 + s3)

c = np.trace(sig_yy) - np.trace(sig_yx @ np.linalg.pinv(sig_xx) @ sig_yx.T)

print(t1 + t2 + c)

6.097788
6.177645
